In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'control': 0, '100nM': 1}

# Load graph dataset and process if neede
graph_path = data_dir / '13cyc_3D' / 'graphs' 

dataset = PPIGraph.GraphDataset2D3D(graph_path, 'raw', '2D3D_pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset2D3D(2492):
Number of graphs: 2492
Number of features: 13
Number of classes: 2
Train set: 1197, test set: 996, val set: 299
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 42358], pos=[3475, 2], labels=[3475, 13], nuclei=[3475], weight=[42358], condition=[32], fov=[32], id=[32], pos3D=[3475, 3], edge_attr=[42358, 1], x=[3475, 13], y=[32], edge_weight_2D=[42358], edge_weight_3D=[42358], name=[32], batch=[3475], ptr=[33])



In [9]:
# # Print some information on the dataset
# print(f'Dataset: {dataset}:')
# print('======================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
# for step, data in enumerate(train_loader):

#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()
#     break

In [10]:
data.edge_weight_2D

tensor([0.0636, 0.1941, 0.3601,  ..., 0.0760, 0.5264, 0.2015])

In [11]:
data.edge_weight_3D

tensor([5.4126e-01, 3.5844e-02, 1.0574e-05,  ..., 7.2026e-01, 8.6688e-01,
        8.1299e-01])

# K-fold on filter dataset

In [12]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [14]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 50

graph_path = data_dir / '13cyc_3D' / 'graphs' 
dataset = PPIGraph.GraphDataset2D3D(graph_path, 'raw', '2D3D_pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [15]:
len(dataset_filtered)

2153

In [16]:
dataset_name = '13PPI_2D3D_fusion'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / '13cyc_3D' /"saved_models_fusion" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01122024_{dataset_name}_{condition}'

In [17]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)
epochs = 65

# NUM_LAYERS = 2
# HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']
# num_layers = [2,3,4]
# hiddens = [16, 32, 64]


# # model = 'GAT'
# model = 'GINConv'
params = [[2,64,'attention','MLP'], 
[2,16,'attention','GraphConv'],
[2,64,'attention2','GCN'],
[3,64,'attention','GAT'],
[3,16,'mean','GINConv']]

Global seed set to 42


In [18]:
params

[[2, 64, 'attention', 'MLP'],
 [2, 16, 'attention', 'GraphConv'],
 [2, 64, 'attention2', 'GCN'],
 [3, 64, 'attention', 'GAT'],
 [3, 16, 'mean', 'GINConv']]

In [19]:
import PPIGraph
import torch
torch.set_float32_matmul_precision('medium')

In [20]:
fusion = 'concat'

In [22]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in params:
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fusion}_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold_fusion(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=13,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                batch_size=128,
                                                fusion=fusion,
                                                graph_pooling=pool)
        run.finish()


y:\coskun-lab\Thomas\23_PLA_revision\data\13cyc_3D\saved_models_fusion\13PPI_2D3D_fusion\Graph_GNNs_Kfold\MLP_2_64_concat_0\attention\GraphLevelMLP\GraphLevelMLP.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\13cyc_3D\saved_models_fusion\13PPI_2D3D_fusion\Graph_GNNs_Kfold\GraphConv_2_16_concat_0\attention\GraphLevelGraphConv\GraphLevelGraphConv.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\13cyc_3D\saved_models_fusion\13PPI_2D3D_fusion\Graph_GNNs_Kfold\GCN_2_64_concat_0\attention2\GraphLevelGCN\GraphLevelGCN.ckpt


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | GAT              | 44.6 K
2  | model3D     | GAT              | 44.6 K
3  | head        | Sequential       | 8.4 K 
4  | loss_module | CrossEntropyLoss | 0     
5  | train_acc   | BinaryAccuracy   | 0     
6  | train_auroc | BinaryAUROC      | 0     
7  | train_f1    | BinaryF1Score    | 0     
8  | valid_acc   | BinaryAccuracy   | 0     
9  | valid_auroc | BinaryAUROC      | 0     
10 | valid_f1    | BinaryF1Score    | 0     
11 | pool2D      | GlobalAttention  | 65    
12 | pool3D      | GlobalAttention  | 65    
--------------------------------------------------
97.7 K    Trainable params
0 

Using GAT


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchmetrics\utilities\prints.py:42: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchmetrics\utilities\prints.py:42: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
`Trainer.fit` stopped: `max_epochs=65` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▇▅▆▆▇▇▇▆▆▇▇▇▇▇▇▇▇▇█▇▇███▇▇█▇████████
train_auc,▁▃▅▆▆▅▆▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███▇████
train_f1,▁▅▆▇▇▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇█▇▇███▇██▇██████▇█
train_loss_epoch,█▅▄▄▃▄▃▃▃▂▂▃▃▃▂▂▂▂▂▃▂▂▁▂▂▁▂▁▂▂▂▂▁▁▁▂▁▁▁▁
train_loss_step,██▆▇▃▄▄▇▄▁▅▃▅▆▅▅▅▃▅▅▄▆▂▃▅▁▄▅▃▂▄▄▂▃▃▇▂▅▆▄
trainer/global_step,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▄▂▂▂▂▂▂▅▅▅▅▆▆▂▂▂▂▃▃▇▇▇███▃
val_acc,▁▂▆▇▇▇▆█▇▆█▆▇▇▇▇▇▆█▇▆▇████▇█▆▇█▇▇▇█▇▇█▆▇
val_auc,▁▂▃▃▄▅▄▄▅▅▆▅▅▅▆▄▅▅▅▃▅▆▆▇▇▇▆▇▄▂█▇▇▅█▆▆▇▄▅
val_f1,▁▂▆▆▇▇▆▇▇▆▇▆▆▇▇▇▆▆▇▇▆▇████▇█▅▆▇▇▇▇█▇▇█▅▇


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▅▆▆▆▆▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇██▇█▇████
train_auc,▁▄▅▆▆▆▆▇▆▇▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▇▇██▇█████████
train_f1,▁▅▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇█▇█▇▇██▇██▇██████
train_loss_epoch,█▅▄▃▃▃▃▂▃▂▃▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁
train_loss_step,█▆▆▅▇▇▇▆▅▂▄▅▆▆▄▂▄▆▅▆▇▃▄▁█▅▆▁▃▂▃▆▃▁▄▄▇▃▃▃
trainer/global_step,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▄▂▂▂▂▂▂▅▅▅▅▆▆▂▂▂▂▃▃▇▇▇███▃
val_acc,▁▁▆▇▇▇▇▇▇██▇▇▇▇▇▇█▇▆▇█████▇█▇▇████▇██▇▇█
val_auc,▁▂▃▄▅▄▄▆▆▆▅▆▆▆▅▅▆▆▅▆▆▆▆▆▇▇▆▆▇▆▇█▇▇▇██▇██
val_f1,▁▁▆▆▇▆▅▇▇▇▇▆▇▇▆▇▆█▇▁▇██▇▇▇▆▇▆▇██▇█▅▇▇▅▅▇


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\13cyc_3D\saved_models_fusion\13PPI_2D3D_fusion\Graph_GNNs_Kfold\GINConv_3_16_concat_4\mean\GraphLevelGINConv exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | GIN              | 1.7 K 
2  | model3D     | GIN              | 1.7 K 
3  | head        | Sequential       | 562   
4  | loss_module | CrossEntropyLoss | 0  

GINConv


`Trainer.fit` stopped: `max_epochs=65` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇███▇█████
train_auc,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇████▇█▇█████████
train_f1,▁▄▅▆▆▆▇▇▇▇▇▆▇▆▇▇▇▇▇▇▇▆▇▇▇█▇▇▇▇▇██▇▇██▇██
train_loss_epoch,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▂▁▁
train_loss_step,█▅▅▄▄▃▅▃▃▅▃▁▃▆▅▂▂▂▃▃▂▃▃▃▅▃▃▂▂▃▄▄▂▃▂▂▁▂▂▁
trainer/global_step,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▄▂▂▂▂▂▂▅▅▅▅▆▆▂▂▂▂▃▃▇▇▇███▃
val_acc,▁▂▄▃▄▆▅▅▆▅▆▄▆▅▆▅▆▆▆▇▄▆▄▅▆▇▄▄▆▅▅▇▆▅▇█▇▆▆▄
val_auc,▁▃▄▄▅▇▆▆▆▅▇▆▇▇▆▇▆▇█▆▅▆▇▆▆▇▆▆▇▇▅▇▇▆▇██▇▇▆
val_f1,▁▄▅▂▄▆▆▅▆▅▆▄▇▃▅▅▅▆▆▆▅▅▃▅▆▆▃▆▆▅▅▇▅▄▇█▆▅▆▅
